In [7]:
import numpy as np 
import pandas as pd

In [8]:
data = pd.read_csv('phone_data_cleaned.csv')

In [13]:
data.head()

,model,price,rating,company,is_dual_sim,has_5g,supports_volte,supports_wifi,supports_nfc,ram_amount,...,battery_life,charging_speed,screen_size,resolution_width,resolution_height,memory_card_supported,primary_camera_resolution,num_rear_cameras,clock_speed_ghz,screen_size_category
0,OnePlus 11 5G,54999.0,89.0,OnePlus,1,1,1,1,1,12,...,5000.0,100.0,6.70,1440.0,3216.0,0,50.0,3,3.2,large
1,OnePlus Nord CE 2 Lite 5G,19989.0,81.0,OnePlus,1,1,1,1,0,6,...,5000.0,33.0,6.59,1080.0,2412.0,1,64.0,3,2.2,large
2,Samsung Galaxy A14 5G,16499.0,75.0,Samsung,1,1,1,1,0,4,...,5000.0,15.0,6.60,1080.0,2408.0,1,50.0,3,2.4,large
3,Motorola Moto G62 5G,14999.0,81.0,Motorola,1,1,1,1,0,6,...,5000.0,33.0,6.55,1080.0,2400.0,1,50.0,3,2.2,large
4,Realme 10 Pro Plus,24999.0,82.0,Realme,1,1,1,1,0,6,...,5000.0,67.0,6.70,1080.0,2412.0,0,108.0,3,2.6,large


In [14]:
# Collect customer input
min_price = float(input("Enter minimum price: "))
max_price = float(input("Enter maximum price: "))

# Display available brands
available_brands = data['company'].unique()
print("Available brands: ", ", ".join(available_brands))
preferred_brands_input = input("Enter preferred brands (comma-separated): ").strip().split(',')
preferred_brands = [brand.strip() for brand in preferred_brands_input]

# Collect screen size preference
screen_size_preference = input("Preferred screen size (large/medium/small): ").strip().lower()

Enter minimum price: 1
Enter maximum price: 1000000
Available brands:  OnePlus, Samsung, Motorola, Realme, Apple, Xiaomi, Nothing, Oppo, Vivo, OPPO, Poco, iQOO, Jio, Tecno, Tesla, Google, Infinix, Cola, POCO, iKall, LeEco, Lava, Honor, Nubia, Redmi, Asus, Oukitel, Nokia, Vertu, BLU, Sony, Lyf, Huawei, ZTE, Lenovo, Micromax, Letv, itel, LG, Gionee, Leitz, Doogee, Sharp, Blackview
Enter preferred brands (comma-separated): Samsung
Preferred screen size (large/medium/small): large


In [15]:
# Filter based on price range
filtered_phones = data[
    (data['price'] >= min_price) & 
    (data['price'] <= max_price)
]

# Check if there was any input for preferred brands
if preferred_brands_input and preferred_brands_input[0].strip().lower() != 'none':
    filtered_phones = filtered_phones[filtered_phones['company'].isin(preferred_brands)]
else:
    # Include all brands
    filtered_phones = filtered_phones

# Filter based on screen size preference
filtered_phones = filtered_phones[filtered_phones['screen_size_category'] == screen_size_preference]

In [16]:
# Assign weights to features
battery_weight = 0.25
camera_weight = 0.2
screen_weight = 0.15
charging_weight = 0.1
ram_weight = 0.1
storage_weight = 0.1
nfc_weight = 0.05
volte_weight = 0.05

# Calculate the score for each phone
filtered_phones['score'] = (
    filtered_phones['battery_life'] * battery_weight + 
    filtered_phones['primary_camera_resolution'] * camera_weight + 
    filtered_phones['screen_size'] * screen_weight + 
    filtered_phones['charging_speed'] * charging_weight + 
    filtered_phones['ram_amount'] * ram_weight + 
    filtered_phones['inbuilt_storage'] * storage_weight + 
    filtered_phones['supports_nfc'] * nfc_weight + 
    filtered_phones['supports_volte'] * volte_weight
)

# Rank the phones based on the score
filtered_phones = filtered_phones.sort_values(by='score', ascending=False)

In [17]:
# Number of recommendations
top_n = 5

# Select the top N recommended phones
recommended_phones = filtered_phones.head(top_n)

# Display the recommended phones
print("Top recommended phones based on your preferences:")
print(recommended_phones[['model', 'price', 'battery_life', 'primary_camera_resolution', 'screen_size', 'charging_speed', 'score']])

Top recommended phones based on your preferences:
                      model    price  battery_life  primary_camera_resolution  \
716      Samsung Galaxy M62  23999.0        7000.0                       64.0   
417      Samsung Galaxy F63  21999.0        7000.0                       64.0   
786      Samsung Galaxy M35  22999.0        6000.0                       64.0   
696  Samsung Galaxy M51s 5G  24990.0        6000.0                       64.0   
134   Samsung Galaxy M34 5G  16999.0        6000.0                       64.0   

     screen_size  charging_speed      score  
716         6.71            33.0  1780.6065  
417         6.70            33.0  1780.6050  
786         6.50            33.0  1530.7750  
696         6.70            33.0  1530.5550  
134         6.50            25.0  1529.7750  
